In [ ]:
# Anselmus Pavel Adriska - 2301851414
# UAS CV Nomor 2C

import os
import cv2
import numpy as np
import pandas as pd
from skimage import feature
from matplotlib import pyplot as plt
from skimage.exposure import histogram
from scipy.spatial.distance import euclidean

In [ ]:
other_hist = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        try:
            img_path = os.path.join(dirname, filename)
            img_path = img_path.replace('/kaggle','..')
            img = cv2.imread(img_path)
            img = cv2.resize(img, (300, 300), interpolation=cv2.INTER_AREA)
            img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            
#             Gunakan salah satu algoritma di bawah
#             Robert
            kernel_x = np.array([[1, 0], [0, -1]])
            kernel_y = np.array([[0, 1], [-1, 0]])
            robert_x = cv2.filter2D(img_gray, -1, kernel_x)
            robert_y = cv2.filter2D(img_gray, -1, kernel_y)
            processed_img_gray = cv2.addWeighted(robert_x, 0.5, robert_y, 0.5, 0)
            
#             Canny
#             processed_img_gray = cv2.Canny(img_gray, 100, 200)
            
#             Prewit
#             prewitt_x = cv2.filter2D(img_gray, -1, kernel_x)
#             prewitt_y = cv2.filter2D(img_gray, -1, kernel_y)
#             processed_img_gray = cv2.addWeighted(prewitt_x, 0.5, prewitt_y, 0.5, 0)
            
            lbp = feature.local_binary_pattern(processed_img_gray, 24, 8, method='uniform')
            n_bins = int(lbp.max()+1)
            hist,_ = histogram(lbp, n_bins, normalize=True, channel_axis=None)
            
            other_hist.append([img, hist])
        except:
            continue

In [ ]:
test_img_path = '../input/indonesian-batik-motifs/batik-sidomukti/1.jpg'
test_img = cv2.imread(test_img_path)
test_img = cv2.resize(test_img, (300, 300), interpolation=cv2.INTER_AREA)
test_img_gray = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)

# Gunakan salah satu algoritma di bawah
#Robert
kernel_x = np.array([[1, 0], [0, -1]])
kernel_y = np.array([[0, 1], [-1, 0]])
robert_x = cv2.filter2D(test_img_gray, -1, kernel_x)
robert_y = cv2.filter2D(test_img_gray, -1, kernel_y)
processed_test_img_gray = cv2.addWeighted(robert_x, 0.5, robert_y, 0.5, 0)

#Canny
# processed_test_img_gray = cv2.Canny(test_img_gray, 100, 200)

#Prewitt
# kernel_x = np.array([[1, 1, 1], [0, 0, 0], [-1, -1, -1]])
# kernel_y = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]])
# prewitt_x = cv2.filter2D(test_img_gray, -1, kernel_x)
# prewitt_y = cv2.filter2D(test_img_gray, -1, kernel_y)
# processed_test_img_gray = cv2.addWeighted(prewitt_x, 0.5, prewitt_y, 0.5, 0)

lbp = feature.local_binary_pattern(processed_test_img_gray, 24, 8, method='uniform')
n_bins = int(lbp.max()+1)
test_hist,_ = histogram(lbp, n_bins, normalize=True, channel_axis=None)

In [ ]:
result = []
for img, hist in other_hist:
    distance = euclidean(test_hist, hist)
    result.append((distance, img))

In [ ]:
fig = plt.figure()
fig.add_subplot(1,1,1)
plt.imshow(cv2.cvtColor(processed_test_img_gray,cv2.COLOR_BGR2RGB))
plt.title("Citra Uji")
plt.show()

def sortFunc(e):
    return e[0]

result.sort(key=sortFunc)

i=0
for j in range(1,3):
    fig = plt.figure(figsize=(15,15))
    for k in range(0,5):
        plt.subplot(j,5,k+1)
        plt.imshow(cv2.cvtColor(result[i][1],cv2.COLOR_BGR2RGB))
        plt.title(round(result[i][0],4))
        i+=1
    plt.show()